In [1]:
import json
import base64
import pandas as pd
import s3fs
import os
import boto3
s3_client = boto3.client('s3', region_name='us-east-2')

In [2]:
fs = s3fs.S3FileSystem()

In [3]:
file = fs.ls('s3:...')  #change the path here
json_file = file[1:]

In [4]:
df = pd.DataFrame(json_file, columns = ['id'])
#df

In [5]:
import json
import boto3
scores = []
results = []
for file in json_file:
    data= s3_client.get_object(Bucket='ingo-images', Key=file[12:])
    client = data['Body'].read().decode()
    json_data = json.loads(client)
    a = json_data['predictions'][0]['classes']
    results.append(a)
    b = json_data['predictions'][0]['scores']
    scores.append(b)

In [6]:
df_results = pd.DataFrame(results, columns = ['results1','results2','results3','results4']) 
#df_results

In [7]:
my_list_a_df = pd.DataFrame(scores, columns = ['scores1','scores2','scores3','scores4'])
#my_list_a_df

In [8]:
df_final_score = pd.concat([df,df_results], axis = 1)
df_final_final_score = pd.concat([df_final_score,my_list_a_df], axis=1)

In [9]:
arr_classify = pd.Series([None]*len(df), index=df_final_final_score.index)
arr_prob = pd.Series([None]*len(df), index=df_final_final_score.index)

In [10]:
for index, row in df_final_final_score.iterrows():
    if row["results1"]=="Crystals" and row["scores1"]>=0.85:
        arr_classify[index]=1
    else:
        arr_classify[index]=0
    index_crystal = 0
    for i in range(1, 5):
        if row[i]=="Crystals":
            index_crystal = i
            break
    arr_prob[index] = row[index_crystal+4]

In [11]:
df_final_final_score["classify"] = arr_classify
df_final_final_score["prob"] = arr_prob

In [12]:
df_final = df_final_final_score[["id","classify","prob"]]

In [13]:
df_final.to_excel("json_result_xdmj.xlsx")